# Databases

In [1]:
import pandas as pd
import json

In [2]:
# import data
articoli = pd.read_csv('data/lista_articoli.csv')
filtrato = pd.read_csv('data/storico_filtrato.csv', decimal=',')
ordini = pd.read_csv('data/storico_ordini.csv')
pianificazione = pd.read_csv('data/storico_pianificazione.csv')


In [3]:
# sistema storico filtrato

df = filtrato.copy()

df.drop(columns='N° MC', inplace=True)
df.rename(columns={
    'Unnamed: 1': 'anno inizio',
    'Unnamed: 2': 'mese articolo',
    'Unnamed: 3': 'N_mc'
    }, 
    inplace=True
)
df.dropna(subset=['data ora partenza'], inplace=True)

df['anno inizio'] = df['data ora partenza'].apply(lambda x: x.split(' ')[0].split('/')[2])
df['mese articolo'] = df['data ora partenza'].apply(lambda x: x.split(' ')[0].split('/')[1])

filtrato = df.copy()
filtrato.head()

,anno inizio,mese articolo,N_mc,cod articolo,descrizione,N° fusi reali,velocita fusi inferiori,data ora partenza,data ora fine,data ora fine manuale,ore lavorate dalla partenza,KWh assorbiti,KW medi,produzione [kg],KWh/Kg,€/Kg,note,N° scheda
2,2024,04,1.0,2277DN mini,"Ly22/8f5 dc tpm 2100 st. 3,4 (DE)",NaN,20194.0,11/04/2024 14:20:33,29/04/2024 21:59:49,NaN,439.48,5921.32,13.4735,121.47,48.74,10.72,x la 7°levata vedere se mettere lycra 22 usata...,NaN
3,2024,04,2.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",NaN,21112.0,03/04/2024 13:41:13,17/04/2024 04:36:25,NaN,326.78,5583.02,17.0850,135.82,41.11,9.04,NaN,NaN
4,2024,04,3.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",448.0,21026.0,24/04/2024 12:34:52,08/05/2024 07:06:28,NaN,330.38,5914.74,17.9028,137.49,43.02,9.46,NaN,NaN
5,2024,04,4.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",NaN,20922.0,19/04/2024 11:01:44,06/05/2024 19:42:55,NaN,416.55,7735.80,18.5711,173.13,44.68,9.83,NaN,NaN
6,2024,04,5.0,2271DN 20700 mini,"K22/8f5 dc tpm 3000 st. 2,8 (CI)",448.0,21077.0,26/04/2024 13:48:42,21/05/2024 15:32:12,NaN,601.68,9070.20,15.0748,152.78,59.37,13.06,\n\n,23/70404-M8


## Macchine x Prodotti

In [54]:
# clean data
df = filtrato.loc[:, ['N_mc', 'cod articolo']]
df = df.rename(columns={'Unnamed: 3': 'N_mc'})
df = df.dropna(subset=['N_mc'])

# remove duplicates and sort by N_mc
unique_df = df.drop_duplicates(subset=['N_mc', 'cod articolo'])
macchine_df = unique_df.sort_values(by=['N_mc'])

macchine_df.head()

,N_mc,cod articolo
2,1.0,2277DN mini
3,2.0,2011DN mini
4,3.0,2011DN mini
5,4.0,2011DN mini
441,5.0,2011DN mini


In [55]:
# dictionary from df
df = macchine_df

macchine = {}
for key in df['N_mc'].unique():
    values = [v for v in df[df['N_mc'] == key].loc[:, 'cod articolo'].values]
    macchine[int(key)] = values

# save to json
with open('data/macchine.json', 'w') as f:
    json.dump(macchine, f, skipkeys=True, ensure_ascii=False, indent=4)

# load from json
with open('data/macchine.json', 'r') as f:
    macchine = json.load(f)

macchine


{'1': ['2277DN mini'],
 '2': ['2011DN mini'],
 '3': ['2011DN mini'],
 '4': ['2011DN mini'],
 '5': ['2011DN mini', '2271DN 20700 mini'],
 '6': ['749DN mini'],
 '7': ['X22-1703DN minispole', '2277DN mini'],
 '8': ['2271DN 20700 mini'],
 '9': ['2271DN 20700 mini'],
 '10': ['2271DN 20700 mini'],
 '11': ['2271DN 20700 mini'],
 '12': ['2271DN 20700 mini', '2271DN 20000 mini'],
 '13': ['X23-3399S+ZF'],
 '14': ['4428ZN/F', 'manilas+z', '2270s+z rpm 19800'],
 '15': ['2270S+ZN 19800'],
 '16': ['2270S+ZN 19800', '2270s+z'],
 '17': ['2270s+z', '2270S+ZN 19800', '7034ZN'],
 '18': ['7032S+ZN', 'X23-2299V5ZF', '7032s+z', '4466S+ZN'],
 '19': ['2270S+ZN 19800', '2270s+z'],
 '20': ['2270S+ZN 19800'],
 '21': ['2270s+z rpm 21000',
  "X21-1796ZN ''la mc è in s+z utilizzato solo piano inf",
  '4412SN',
  '7036ZN'],
 '22': ['X22-7810s+z', '10353ZF', 'X22-7810S-ZN354 4 ci', '2250S+ZN'],
 '23': ['31002DN1'],
 '24': ['X24-4401S+ZF2', 'X23-4493S+ZF'],
 '25': ['X23-4493S+ZF', 'X21-2291sf', 'X21-2291SF'],
 '26': [

## Prodotti x Macchine

In [57]:
df = filtrato.loc[:, ['N_mc', 'cod articolo']]
df.columns = ['N_mc', 'cod articolo']

# switch columns
df = df[['cod articolo', 'N_mc']]
df.dropna(inplace=True)
df.head()

articoli_df = df

articoli = {}
for key in df['cod articolo'].unique():
    values = [int(v) for v in df[df['cod articolo'] == key].loc[:, 'N_mc'].unique()]
    articoli[key] = values

with open('data/articoli.json', 'w') as f:
    json.dump(articoli, f, skipkeys=True, ensure_ascii=False, indent=4)

articoli = json.load(open('data/articoli.json'))
articoli

{'2277DN mini': [1, 7],
 '2011DN mini': [2, 3, 4, 5],
 '2271DN 20700 mini': [5, 8, 9, 10, 11, 12, 47, 48, 62, 63, 64, 65, 66, 61],
 '749DN mini': [6, 49, 50],
 'X22-1703DN minispole': [7],
 'X23-3399S+ZF': [13, 44],
 '2270s+z rpm 19800': [14],
 '2270S+ZN 19800': [15, 16, 17, 19, 20, 26],
 'X23-2299V5ZF': [18, 72],
 '7032S+ZN': [18],
 '2270s+z rpm 21000': [21],
 '2250S+ZN': [22],
 '31002DN1': [23],
 'X24-4401S+ZF2': [24],
 'X23-4493S+ZF': [24, 25],
 'X21-2291SF': [25],
 'X21-2291sf': [25],
 '2270S+ZN mini': [26],
 'X21-2099S+ZN mini': [27, 28, 33, 34],
 'X24-2001S+ZN': [28, 60],
 '13007ZN/F-14500': [29, 30, 32, 43],
 '28503ZN': [29],
 '2246ZN 10800': [31, 44],
 'X23-2280S+ZN': [35],
 'X20-2222DN2 mini': [36],
 '33002DN': [37, 41],
 's22-700011DN': [37],
 'S600A04DN': [37],
 'S600A03DN': [37],
 '33001DN2': [38, 41, 67],
 'X23-8397DN': [38],
 '31003DN220274': [39, 40],
 'X21-39599V7DN0332': [39],
 '62002DN': [39],
 '39502DN-14446': [39],
 '39502dn': [39],
 '62002DN2': [39],
 'X21-20099DN2

## Prodotti
Articoli: macchine, (gruppo_macchine), n_levate (= no_cicli), n_fusi (joint con macchine, check ambigui), tempo_levata, kg_levata (kg_ora x ore_levata)
- toglio ambigui

### Macchine x caratteristiche

In [28]:
groups_filename= "data/Gruppi_Macchine_per_articoli.json"

In [144]:
with open(groups_filename, 'r') as f:
    groups_file = json.load(f)

divisione = groups_file['Divisione per articoli prodotti']
divisione.keys()

# elimina sottogruppi
groups = {}

for key, group_dict in divisione.items():
    if key + 'a' in group_dict.keys():
        groups[key + 'a'] = group_dict[key + 'a']
    if key + 'b' in group_dict.keys():
        groups[key + 'b'] = group_dict[key + 'b']
    else:
        groups[key] = group_dict

print(groups.keys())

# crea dataframe macchine: caratteristiche
data = []

for classe, group in groups.items():
    
    n_fusi = group.pop('Numero Fusi')
    
    for mode, mode_dict in group.items():
        descrizione = mode_dict.pop('Descrizione')
        macchine = mode_dict.pop('Macchine')

        for m in macchine:
            data.append([m, classe, n_fusi, mode, descrizione])

df = pd.DataFrame(data, columns=['N_mc', 'classe', 'n_fusi', 'mode', 'descrizione'])
df.sort_values(by='N_mc', inplace=True)

# save to json
df.to_json('data/macchine.json', orient='records', indent=4)


dict_keys(['Gruppo1a', 'Gruppo1b', 'Gruppo2', 'Gruppo3a', 'Gruppo3b', 'Gruppo4a', 'Gruppo4b', 'Gruppo5a', 'Gruppo5b', 'Gruppo6'])


# Statistiche

In [67]:
filtrato.head()


,anno inizio,mese articolo,N_mc,cod articolo,descrizione,N° fusi reali,velocita fusi inferiori,data ora partenza,data ora fine,data ora fine manuale,ore lavorate dalla partenza,KWh assorbiti,KW medi,produzione [kg],KWh/Kg,€/Kg,note,N° scheda
2,2024,04,1.0,2277DN mini,"Ly22/8f5 dc tpm 2100 st. 3,4 (DE)",NaN,20194.0,11/04/2024 14:20:33,29/04/2024 21:59:49,NaN,439.48,5921.32,13.4735,121.47,48.74,10.72,x la 7°levata vedere se mettere lycra 22 usata...,NaN
3,2024,04,2.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",NaN,21112.0,03/04/2024 13:41:13,17/04/2024 04:36:25,NaN,326.78,5583.02,17.0850,135.82,41.11,9.04,NaN,NaN
4,2024,04,3.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",448.0,21026.0,24/04/2024 12:34:52,08/05/2024 07:06:28,NaN,330.38,5914.74,17.9028,137.49,43.02,9.46,NaN,NaN
5,2024,04,4.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",NaN,20922.0,19/04/2024 11:01:44,06/05/2024 19:42:55,NaN,416.55,7735.80,18.5711,173.13,44.68,9.83,NaN,NaN
6,2024,04,5.0,2271DN 20700 mini,"K22/8f5 dc tpm 3000 st. 2,8 (CI)",448.0,21077.0,26/04/2024 13:48:42,21/05/2024 15:32:12,NaN,601.68,9070.20,15.0748,152.78,59.37,13.06,\n\n,23/70404-M8


In [68]:
filtrato.describe()

,N_mc,N° fusi reali,velocita fusi inferiori,ore lavorate dalla partenza,KWh assorbiti,KW medi,produzione [kg],KWh/Kg,€/Kg
count,651.000000,360.000000,647.000000,602.000000,602.000000,602.000000,624.000000,575.000000,575.000000
mean,41.476190,268.880556,15973.378671,163.433538,1927.399485,9.925470,129.325913,15.894991,3.495096
std,18.640253,71.303499,4055.193289,151.716547,2231.308768,3.917078,63.858465,20.232336,4.451992
min,1.000000,24.000000,7000.000000,4.980000,0.120000,0.000300,0.260000,0.000000,0.000000
25%,30.000000,256.000000,13195.500000,58.765000,454.525000,7.743075,92.330000,3.650000,0.805000
50%,40.000000,256.000000,14623.000000,109.335000,1027.235000,10.275650,123.740000,6.680000,1.470000
75%,56.500000,264.000000,19936.000000,228.420000,2723.320000,12.511100,171.005000,26.790000,5.895000
max,75.000000,448.000000,21189.000000,649.520000,11224.170000,19.356400,795.300000,245.500000,54.010000


In [ ]:
filtrato['']